In [20]:
import pymupdf
import pytesseract
import pandas as pd
import cv2
import os
import re
pytesseract.pytesseract.tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract.exe"

import os


In [21]:

def extract_images(filename: str):
    doc = pymupdf.open(filename)
    for page_index in range(len(doc)): # iterate over pdf pages
        page = doc[page_index] # get the page
        image_list = page.get_images()

        # print the number of images found on the page
        if image_list:
            print(f"Found {len(image_list)} images on page {page_index}")
        else:
            print("No images found on page", page_index)

        for image_index, img in enumerate(image_list, start=1): # enumerate the image list
            xref = img[0] # get the XREF of the image
            pix = pymupdf.Pixmap(doc, xref) # create a Pixmap

            if pix.n - pix.alpha > 3: # CMYK: convert to RGB first
                pix = pymupdf.Pixmap(pymupdf.csRGB, pix)

            pix.save("page_%s_image_%s.png" % (page_index, image_index)) # save the image as png
            pix = None
    
extract_images('Data Scrape PDF.pdf')

print()

Found 5 images on page 0
Found 5 images on page 1
Found 5 images on page 2
Found 5 images on page 3
Found 5 images on page 4
Found 5 images on page 5
Found 5 images on page 6
Found 5 images on page 7
Found 5 images on page 8
Found 5 images on page 9
Found 5 images on page 10
Found 5 images on page 11
Found 5 images on page 12
Found 5 images on page 13
Found 5 images on page 14
Found 5 images on page 15
Found 5 images on page 16
Found 5 images on page 17
Found 5 images on page 18
Found 5 images on page 19
Found 5 images on page 20
Found 5 images on page 21
Found 5 images on page 22
Found 5 images on page 23
Found 5 images on page 24
Found 5 images on page 25



In [22]:
def image_preprocessing(filename: str) :
    img = cv2.imread(filename)
    cv2.resize(img, None, fx=1.2, fy=1.2, interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    gray = cv2.medianBlur(gray, 3)
    filename = "{}.png".format(os.getpid())
    cv2.imwrite(filename, gray)
    return filename
for image in os.listdir():
    if image.endswith('.png'):
        image_preprocessing(image)
        

In [23]:
filenames = [f for f in os.listdir() if f.endswith('.png')]
print(sorted(filenames))

['13812.png', 'page_0_image_1.png', 'page_0_image_2.png', 'page_0_image_3.png', 'page_0_image_4.png', 'page_0_image_5.png', 'page_10_image_1.png', 'page_10_image_2.png', 'page_10_image_3.png', 'page_10_image_4.png', 'page_10_image_5.png', 'page_11_image_1.png', 'page_11_image_2.png', 'page_11_image_3.png', 'page_11_image_4.png', 'page_11_image_5.png', 'page_12_image_1.png', 'page_12_image_2.png', 'page_12_image_3.png', 'page_12_image_4.png', 'page_12_image_5.png', 'page_13_image_1.png', 'page_13_image_2.png', 'page_13_image_3.png', 'page_13_image_4.png', 'page_13_image_5.png', 'page_14_image_1.png', 'page_14_image_2.png', 'page_14_image_3.png', 'page_14_image_4.png', 'page_14_image_5.png', 'page_15_image_1.png', 'page_15_image_2.png', 'page_15_image_3.png', 'page_15_image_4.png', 'page_15_image_5.png', 'page_16_image_1.png', 'page_16_image_2.png', 'page_16_image_3.png', 'page_16_image_4.png', 'page_16_image_5.png', 'page_17_image_1.png', 'page_17_image_2.png', 'page_17_image_3.png', 'p

In [24]:
from collections import defaultdict

# Dictionary to keep track of counts for each prefix
prefix_counts = defaultdict(int)

def images_classification(filename: str):
    print(f"Classifying {filename}...")
    text = pytesseract.image_to_string(filename)
    if 'Received by OCD' in text:
        
        new_filename = 'OCD_' + filename
        os.rename(filename, new_filename)
    elif text.startswith('Page'):
        os.remove(filename)
    elif text == '':
        os.remove(filename)
    elif text.startswith('Released'):
        os.remove(filename)
    elif 'APPLICATION CHECKLIST' in text:
        new_filename = 'application_checklist_head_' + filename
        os.rename(filename, new_filename)
    elif 'API No.' in text:
        new_filename = 'Well_data_' + filename
        os.rename(filename, new_filename)
    elif 'LAT' in text and 'LONG' in text:
        new_filename = 'well_location_' + filename
        os.rename(filename, new_filename)
    elif "of Leasehold Interest" in text:
        new_filename = 'ownership_' + filename
        os.rename(filename, new_filename)
    elif "TOTAL WELL COST" in text:
        new_filename = 'expenditure_' + filename
        os.rename(filename, new_filename)
    else:
        os.remove(filename)
        

for filename in os.listdir():
    if filename.endswith('.png'):
        images_classification(filename)


Classifying 13812.png...
Classifying page_0_image_1.png...
Classifying page_0_image_2.png...
Classifying page_0_image_3.png...
Classifying page_0_image_4.png...
Classifying page_0_image_5.png...
Classifying page_10_image_1.png...
Classifying page_10_image_2.png...
Classifying page_10_image_3.png...
Classifying page_10_image_4.png...
Classifying page_10_image_5.png...
Classifying page_11_image_1.png...
Classifying page_11_image_2.png...
Classifying page_11_image_3.png...
Classifying page_11_image_4.png...
Classifying page_11_image_5.png...
Classifying page_12_image_1.png...
Classifying page_12_image_2.png...
Classifying page_12_image_3.png...
Classifying page_12_image_4.png...
Classifying page_12_image_5.png...
Classifying page_13_image_1.png...
Classifying page_13_image_2.png...
Classifying page_13_image_3.png...
Classifying page_13_image_4.png...
Classifying page_13_image_5.png...
Classifying page_14_image_1.png...
Classifying page_14_image_2.png...
Classifying page_14_image_3.png...


In [25]:
df = pd.read_excel('Data Scrape Excel.xlsx', 'Scrape Spreadsheet', )
df.columns

Index(['Case #', 'Date Received By OCD', 'Pooling Approval Date', 'State',
       'County', 'Landing  Zone', 'Operator', 'Pooling Code', 'Well Family',
       'Well Name', 'API #', 'TVD', 'Surface Hole Lat', 'Surface Hole Long',
       'Bottom Hole Lat', 'Bottom Hole Long', 'AFE Total',
       'WI/Mineral/OR Owner', 'WI %', 'Documents'],
      dtype='object')

In [26]:
for filename in os.listdir():
    if filename.startswith("OCD") and filename != "OCD_page_0_images_1.png":
        os.remove(filename)

In [27]:
def extract_data(filenames:list[str]) -> pd.DataFrame:    
    
    well_names =[]
    case_num = []
    ocd = []
    pool_approve = []
    state = []
    county = []
    landing_zone = []
    operator = []
    pooling_code = []
    well_family = []
    apis = []
    tvds = []
    surface_lat = []
    surface_long = []
    bottom_lat = []
    bottom_long = []
    afe = []
    ownership = []
    wi = []
    documents = []
    for filename in filenames:
        print(filename)
        if filename.startswith("OCD"):
            text = pytesseract.image_to_string(filename)
            ocd_dates = re.findall(r"Received\sby\sOCD:\s(\d{1,2}/\d{1,2}/\d{4})", text)
            for ocd_date in ocd_dates:
                ocd.append(ocd_date)
            
        if filename.startswith("Well_data"):
            text = pytesseract.image_to_string(filename)
            well_name = re.findall(r"Red\sHills\sWest\s\d{2}/\d{2}\sFed\.\sCom\.\sWell\sNo\.\s\d{3}H", text)
            for well in well_name:
                print(well)    
                well_names.append(well)
            api = re.findall(r"API\sNo\.\s(\d{2}-\d{3}-(\d{5}|Pending))", text)
            for i in api:
                apis.append(i[0])
            tvd = re.findall(r"TVD\s(\d+)\sfeet", text)
            for num in tvd:
                tvds.append(num)
            print(set(well_names))
            print(api)
            print(tvd)
        if filename.startswith("application_"):
            text = pytesseract.image_to_string(filename)
            pool_codes = re.findall(r"Pool\sCode\s(\d+)", text)
            for pool_code in pool_codes:
                pooling_code.append(pool_code)
            cases = re.findall(r"Case:\s(\d+)", text)
            for case in cases:    
                case_num.append(case)
            locations = re.findall(r"(\w+)\sCounty,\s(\w+\s\w+)", text)
            for location in locations:
                county.append(location[0])
                state.append(location[1])
            well_fams = re.findall( r"Well\sFamily\s*\n\n([A-Za-z\s]+)", text)
            for well_fam in well_fams:
                well_family.append(well_fam.split("\n\n")[0])
            operator_names = re.findall(r"Designated\sOperator\s&\sOGRID\s*\n\(affiliation\sif\sapplicable\)\s*\n\n(.+?)/OGRID\s\d+"
, text)
            for operator_name in operator_names:
                operator.append(operator_name)
            lands_zone = re.findall(r"Vertical\s(.+)", text)
            for land_zone in lands_zone:
                landing_zone.append(land_zone)
            # print(location)
            print(case)
            # print(pool_code)
        if filename.startswith("well_location"):
            text = pytesseract.image_to_string(filename)
            lat_pattern = re.findall(r"LAT:\s([\d.]+)°\s[N|S]", text)
            long_pattern = re.findall(r"LONG:\s([\d.]+)°\s[W|E]", text)
            SURFACE_LAT = lat_pattern[0]
            SURFACE_LONG = long_pattern[0]
            BOTTOM_LAT = lat_pattern[-1]
            BOTTOM_LONG = long_pattern[-1]
            surface_lat.append(SURFACE_LAT)
            surface_long.append(SURFACE_LONG)
            bottom_lat.append(BOTTOM_LAT)
            bottom_long.append(BOTTOM_LONG)
            print(lat_pattern)
            print(long_pattern)
        if filename.startswith("expenditure_"):
            text = pytesseract.image_to_string(filename)
            costs = re.findall(r"TOTAL\sWELL\sCOST\s\$(\d{1,3}(?:,\d{3})*)", text)
            for cost in costs:
                afe.append(cost)
        
        

    dict = {
        'Case #':[], 'Date Received By OCD':[], 'Pooling Approval Date':[], 'State':[],
        'County':[], 'Landing  Zone':[], 'Operator':[], 'Pooling Code':[], 'Well Family':[],
        'Well Name':[], 'API #':[], 'TVD':[], 'Surface Hole Lat':[], 'Surface Hole Long':[],
        'Bottom Hole Lat':[], 'Bottom Hole Long':[], 'AFE Total':[],
        'WI/Mineral/OR Owner':[], 'WI %':[], 'Documents':[]
}    
    well_names=well_names[1:]
    dict['Case #'] = case_num * len(well_names)
    dict['Date Received By OCD'] = ocd * len(well_names)
    dict['Pooling Approval Date'] = pool_approve * len(well_names)
    dict['State'] = state * len(well_names)
    dict['County'] = county * len(well_names)
    dict['Landing  Zone'] = landing_zone *len(well_names)
    dict['Operator'] = operator * len(well_names)
    dict['Pooling Code'] = pooling_code * len(well_names)
    dict['Well Family'] = well_family * len(well_names)
    dict['Well Name'] = well_names
    dict['API #'] = apis
    dict['TVD'] = tvds
    dict['Surface Hole Lat'] = surface_lat
    dict['Surface Hole Long'] = surface_long
    dict['Bottom Hole Lat'] = bottom_lat
    dict['Bottom Hole Long'] = bottom_long
    dict['AFE Total'] = afe
    dict['WI/Mineral/OR Owner'] = ownership
    dict['WI %'] = wi
    dict['Documents'] = documents           

    
    return dict

data = extract_data([file for file in os.listdir() if file.endswith('.png')])
df = pd.DataFrame.from_dict(data, orient='index').transpose()
df.to_csv("temp.csv", index=False)
df

application_checklist_head_page_2_image_5.png
24337
expenditure_page_20_image_5.png
expenditure_page_21_image_5.png
expenditure_page_22_image_5.png
expenditure_page_23_image_5.png
expenditure_page_24_image_5.png
expenditure_page_25_image_5.png
ownership_page_16_image_1.png
Well_data_page_3_image_1.png
Red Hills West 22/15 Fed. Com. Well No. 502H
Red Hills West 22/15 Fed. Com. Well No. 521H
Red Hills West 22/15 Fed. Com. Well No. 571H
Red Hills West 22/15 Fed. Com. Well No. 502H
Red Hills West 22/15 Fed. Com. Well No. 524H
{'Red Hills West 22/15 Fed. Com. Well No. 521H', 'Red Hills West 22/15 Fed. Com. Well No. 502H', 'Red Hills West 22/15 Fed. Com. Well No. 571H', 'Red Hills West 22/15 Fed. Com. Well No. 524H'}
[('30-025-51120', '51120'), ('30-025-Pending', 'Pending'), ('30-025-Pending', 'Pending'), ('30-025-50993', '50993')]
['10521', '11013', '10251', '10555']
Well_data_page_4_image_5.png
Red Hills West 22/15 Fed. Com. Well No. 574H
Red Hills West 22/15 Fed. Com. Well No. 573H
{'Red 

,Case #,Date Received By OCD,Pooling Approval Date,State,County,Landing Zone,Operator,Pooling Code,Well Family,Well Name,API #,TVD,Surface Hole Lat,Surface Hole Long,Bottom Hole Lat,Bottom Hole Long,AFE Total,WI/Mineral/OR Owner,WI %,Documents
0,24337,None,None,New Mexico,Lea,Bone Spring,Mewbourne Oil Company,97903,Red Hills West Bone Spring wells,Red Hills West 22/15 Fed. Com. Well No. 521H,30-025-51120,10521,32.0215572,103.6666865,32.0500022,103.6698576,"8,784,000",None,None,None
1,24337,None,None,New Mexico,Lea,Bone Spring,Mewbourne Oil Company,97903,Red Hills West Bone Spring wells,Red Hills West 22/15 Fed. Com. Well No. 571H,30-025-Pending,11013,32.0215574,103.6666221,32.0500071,103.6666221,"9,678,900",None,None,None
2,24337,None,None,New Mexico,Lea,Bone Spring,Mewbourne Oil Company,97903,Red Hills West Bone Spring wells,Red Hills West 22/15 Fed. Com. Well No. 502H,30-025-Pending,10251,32.0215575,103.6664931,32.0500122,103.6641782,"9,678,900",None,None,None
3,24337,None,None,New Mexico,Lea,Bone Spring,Mewbourne Oil Company,97903,Red Hills West Bone Spring wells,Red Hills West 22/15 Fed. Com. Well No. 524H,30-025-50993,10555,32.0215577,103.6664285,32.0215577,103.6664285,"9,678,900",None,None,None
4,24337,None,None,New Mexico,Lea,Bone Spring,Mewbourne Oil Company,97903,Red Hills West Bone Spring wells,Red Hills West 22/15 Fed. Com. Well No. 574H,30-025-51121,11018,32.0215574,103.6665575,32.0500073,103.6669534,None,None,None,None
5,24337,None,None,New Mexico,Lea,Bone Spring,Mewbourne Oil Company,97903,Red Hills West Bone Spring wells,Red Hills West 22/15 Fed. Com. Well No. 573H,30-025-Pending,11018,32.0215572,103.6667511,32.0500020,103.6699867,None,None,None,None


In [28]:
# filenames = ["Well_data_page_3_image_1.png", "Well_data_page_4_image_5.png"]
# for filename in filenames:
#     text = pytesseract.image_to_string(filename)
#     well_name = re.findall(r"Red\sHills\sWest\s\d{2}/\d{2}\sFed\.\sCom\.\sWell\sNo\.\s\d{3}H", text)
#     for well in well_name:
#         print(well)
#     print("-----------------------------------------------------------------------------------")
#     print(text)
    

In [29]:
# lat=[]
# long=[]
# for filename in os.listdir():
#     if filename.startswith('well_location_'):
#         text = pytesseract.image_to_string(filename)
#         for line in text.split('\n'):
#             if line.startswith("LAT"):
#                 line = line.split(" ")
#                 lat.append(line[1].replace("°", ""))
#             elif line.startswith("LONG"):
#                 line = line.split(" ")
#                 long.append(line[1].replace("°", ""))

        

In [30]:
# apis = []
# text = pytesseract.image_to_string("Well_data_page_3_image_1.png")
# well_name = re.findall(r"Red\sHills\sWest\s\d{2}/\d{2}\sFed\.\sCom\.\sWell\sNo\.\s\d{3}H", text)

# api = re.findall(r"API\sNo\.\s(\d{2}-\d{3}-(\d{5}|Pending))", text)
# for i in api:
#     apis.append(i[0])
# tvd = re.findall(r"TVD\s(\d+)\sfeet", text)

# print(set(well_name))
# print(apis)
# print(tvd)

In [31]:
# files = ["expenditure_page_20_image_5.png","expenditure_page_21_image_5.png","expenditure_page_22_image_5.png","expenditure_page_23_image_5.png","expenditure_page_24_image_5.png","expenditure_page_25_image_5.png",]
# for file in files:
#     text = pytesseract.image_to_string(file)
#     print(text)
#     print("-----------------------------------------------------------------------------------------------------------------")

In [32]:
# def extract_data(filenames:list[str]) -> pd.DataFrame:    
    
#     well_names =[]
#     case_num = []
#     ocd = []
#     pool_approve = []
#     state = []
#     county = []
#     landing_zone = []
#     operator = []
#     pooling_code = []
#     well_family = []
#     apis = []
#     tvds = []
#     surface_lat = []
#     surface_long = []
#     bottom_lat = []
#     bottom_long = []
#     afe = []
#     ownership = []
#     wi = []
#     documents = []
#     for filename in filenames:
#         print(filename)
        
#         text = pytesseract.image_to_string(filename)
#         ocd_dates = re.findall(r"Received\sby\sOCD:\s(\d{1,2}/\d{1,2}/\d{4})", text)
#         for ocd_date in ocd_dates:
#             ocd.append(ocd_date)
        
    
#         well_name = re.findall(r"Red\sHills\sWest\s\d{2}/\d{2}\sFed\.\sCom\.\sWell\sNo\.\s\d{3}H", text)
#         for well in well_name:
#             print(well)    
#             well_names.append(well)
#         api = re.findall(r"API\sNo\.\s(\d{2}-\d{3}-(\d{5}|Pending))", text)
#         for i in api:
#             apis.append(i[0])
#         tvd = re.findall(r"TVD\s(\d+)\sfeet", text)
#         for num in tvd:
#             tvds.append(num)
#         # print(set(well_names))
#         # print(api)
#         # print(tvd)
    
#         pool_codes = re.findall(r"Pool\sCode\s(\d+)", text)
#         for pool_code in pool_codes:
#             pooling_code.append(pool_code)
#         cases = re.findall(r"Case:\s(\d+)", text)
#         for case in cases:    
#             case_num.append(case)
#         locations = re.findall(r"(\w+)\sCounty,\s(\w+\s\w+)", text)
#         for location in locations:
#             county.append(location[0])
#             state.append(location[1])
#         well_fams = re.findall( r"Well\sFamily\s*\n\n([A-Za-z\s]+)", text)
#         for well_fam in well_fams:
#             well_family.append(well_fam.split("\n\n")[0])
#         operator_names = re.findall(r"Designated\sOperator\s&\sOGRID\s*\n\(affiliation\sif\sapplicable\)\s*\n\n(.+?)/OGRID\s\d+"
# , text)
#         for operator_name in operator_names:
#             operator.append(operator_name)
#         lands_zone = re.findall(r"Vertical\s(.+)", text)
#         for land_zone in lands_zone:
#             landing_zone.append(land_zone)
#         # print(location)
#         # print(case)
#         # print(pool_code)
    
#         lat_pattern = re.findall(r"LAT:\s([\d.]+)°\s[N|S]", text)
#         long_pattern = re.findall(r"LONG:\s([\d.]+)°\s[W|E]", text)
#         if lat_pattern and long_pattern:
#             SURFACE_LAT = lat_pattern[0]
#             SURFACE_LONG = long_pattern[0]
#             BOTTOM_LAT = lat_pattern[-1]
#             BOTTOM_LONG = long_pattern[-1]
#             surface_lat.append(SURFACE_LAT)
#             surface_long.append(SURFACE_LONG)
#             bottom_lat.append(BOTTOM_LAT)
#             bottom_long.append(BOTTOM_LONG)
#         # print(lat_pattern)
#         # print(long_pattern)
    
#         costs = re.findall(r"TOTAL\sWELL\sCOST\s\$(\d{1,3}(?:,\d{3})*)", text)
#         for cost in costs:
#             afe.append(cost)
        
        

#     dict = {
#         'Case #':[], 'Date Received By OCD':[], 'Pooling Approval Date':[], 'State':[],
#         'County':[], 'Landing  Zone':[], 'Operator':[], 'Pooling Code':[], 'Well Family':[],
#         'Well Name':[], 'API #':[], 'TVD':[], 'Surface Hole Lat':[], 'Surface Hole Long':[],
#         'Bottom Hole Lat':[], 'Bottom Hole Long':[], 'AFE Total':[],
#         'WI/Mineral/OR Owner':[], 'WI %':[], 'Documents':[]
# }    
#     well_names=well_names[1:]
#     dict['Case #'] = case_num * len(well_names)
#     dict['Date Received By OCD'] = ocd * len(well_names)
#     dict['Pooling Approval Date'] = pool_approve * len(well_names)
#     dict['State'] = state * len(well_names)
#     dict['County'] = county * len(well_names)
#     dict['Landing  Zone'] = landing_zone *len(well_names)
#     dict['Operator'] = operator * len(well_names)
#     dict['Pooling Code'] = pooling_code * len(well_names)
#     dict['Well Family'] = well_family * len(well_names)
#     dict['Well Name'] = well_names
#     dict['API #'] = apis
#     dict['TVD'] = tvds
#     dict['Surface Hole Lat'] = surface_lat
#     dict['Surface Hole Long'] = surface_long
#     dict['Bottom Hole Lat'] = bottom_lat
#     dict['Bottom Hole Long'] = bottom_long
#     dict['AFE Total'] = afe
#     dict['WI/Mineral/OR Owner'] = ownership
#     dict['WI %'] = wi
#     dict['Documents'] = documents           

    
#     return dict

# data = extract_data([file for file in os.listdir() if file.endswith('.png')])
# df = pd.DataFrame.from_dict(data, orient='index').transpose()
# df.to_csv("temp.csv", index=False)
# df


In [33]:
for filename in os.listdir():
    if filename.endswith('.png'):
        os.remove(filename)